In [6]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from PIL import Image
import os

In [7]:
cwd = os.getcwd()

# loading training data
x_train = []
for i in range(1500):
    im = Image.open(cwd + '/training/{}.tif'.format(i))
    x_train.append(np.asarray(im))
x_train = np.array(x_train)

y_train = np.genfromtxt(cwd + '/labels_training.csv', delimiter=',', skip_header = 1)
y_train = y_train.T[1]

# loading testing data
x_test = []
for i in range(1500,2058):
    im = Image.open(cwd + '/testing/{}.tif'.format(i))
    x_test.append(np.asarray(im))
x_test = np.array(x_test)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhuangdiezhou/Documents/Course Spring 2019/Principles of Machine Learning/Kaggle/training/0.tif'

In [ ]:
# Download model weights
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

# Re-structure model
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024, activation='relu')(x) #dense layer 2
x=Dense(512, activation='relu')(x) #dense layer 3
preds=Dense(1, activation='sigmoid')(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
'''
for i,layer in enumerate(model.layers):
    print(i,layer.name)
'''
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
# training parameters
batch_size = 50
epochs = 30

# defining class weights by different number of classes
class_weight={1: (len(y_train) - np.sum(y_train)) / np.sum(y_train),
              0: 1}

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy',f1,auc])

model.fit(x_train, y_train,
          epochs=epochs,
          batch_size=batch_size,
          class_weight=class_weight,
          verbose=1,
          validation_split = 0.2)

# The fitting process will take about 20~40 min

In [5]:
# Predict and writting CSVs
y = model.predict(x_test)
df = pd.DataFrame({'id':np.arange(1500, 2058, 1),
                  'score':y.T[0]})
df.to_csv('submission.csv', index=False)

NameError: name 'model' is not defined